### Video display in live

In [32]:
from ipywidgets import Video
from timeit import default_timer as timer
from datetime import timedelta

In [33]:
path = '/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv'

In [36]:
start = timer()
video = Video.from_file(path)
end = timer()
print(timedelta(seconds=end-start))

0:00:00.971988


In [37]:
video

Video(value=b'\x1aE\xdf\xa3\x01\x00\x00\x00\x00\x00\x00#B\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\…

The time necessary to load the video is quite high. More than 1 minute. Once it is loaded, it is easy to watch the video.

### Live test of prediction actions

In [1]:
import numpy as np
import sqlite3 as sql
import os
from timeit import default_timer as timer
from datetime import timedelta
from FeaturesData import getPathFeatures, getFeatures, getFeaturesWindow
from math import *
from Network import Network

#from LatenceMeasure import displayMeasureTime, measureTimeWindows

In [52]:
# Select a footblall game in the database
db_path = "/scratch/users/mdelabrassinne/Database/SoccerDB.db"
game = '/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv'

features = getPathFeatures(game)
nms_windows = [20 for i in classes]

/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_baidu_soccer_embeddings.npy


In [69]:
import copy
import json
import os
import numpy as np
import torch

class Results:

    def __init__(self, dir_model, features, window_size, classes, nms_window,fps=1):
        self.dir_model = dir_model
        self.features = features
        self.window_size = window_size
        self.nb_classes = len(classes)
        self.classes = classes
        self.nms_window = nms_window
        self.fps = fps

    def getResults(self, disp_path, conf_path):
        """
        Compute the results of the model from a sequence of features ( of a given size: window_size).
        The results are saved in a folder named "results" in the directory of the model.

        Args:
            param dir_model: directory of the model

        """
        dir_results = "/scratch/users/mdelabrassinne/Results_Test"

        conf_mod_path = os.path.join(self.dir_model, conf_path)
        disp_mod_path = os.path.join(self.dir_model, disp_path)
        model = Network(conf_mod_path, disp_mod_path)
        models = model.get_models()

        if not os.path.exists(dir_results):
            os.mkdir(dir_results)
            
        print(f"Directory {dir_results}")

        def compute_confidences(self, features):
            conf = np.zeros((features.shape[0], self.nb_classes))
            # Make the predictions
            for i in range(0, features.shape[0], self.window_size):
                # Make the predictions
                if i+self.window_size >= features.shape[0]:
                    seq = features[-self.window_size:]
                    seq = np.expand_dims(seq, axis=0)
                    seq = torch.Tensor(seq)
                    seq = seq.transpose(1, 2)
                    confidences = (
                        (models[0](seq)).detach().numpy()).transpose()
                    confidences = np.squeeze(confidences, axis=2)
                    confidences = 1 / (1 + np.exp(-confidences))
                    displacements = (
                        (models[1](seq)).detach().numpy()).transpose()
                    displacements = np.squeeze(displacements, axis=2)
                    end = features.shape[0] - i
                    conf[i:] = self.merge_nms(
                        confidences[-end:], displacements[-end:])
                else:
                    seq = features[i:i+self.window_size]
                    seq = np.expand_dims(seq, axis=0)
                    seq = torch.Tensor(seq)
                    seq = seq.transpose(1, 2)
                    confidences = (
                        (models[0](seq)).detach().numpy()).transpose()
                    confidences = np.squeeze(confidences, axis=2)
                    confidences = 1 / (1 + np.exp(-confidences))
                    displacements = (
                        (models[1](seq)).detach().numpy()).transpose()
                    displacements = np.squeeze(displacements, axis=2)
                    conf[i:i +
                         self.window_size] = self.merge_nms(confidences, displacements)
            return conf
    
        pred_files = []
        features = np.load(self.features)       
        conf1 = compute_confidences(self, features)

        # write the results in a json file
        dir_match = os.path.dirname("test")
        json_file_name = self.writeResults(conf1, dir_results, dir_match)
        pred_files.append(json_file_name)
        
        return evaluate(pred_files, labels, version=2)

    def writeResults(self, conf: np.ndarray,  dir_results: str, dir_match: str):
        """
        Write the results in a JSON file.

        Args:
            param conf: confidence of the model
            param dir_results: directory of the results
        """

        def add_action(self, actions: list, confidences: np.ndarray, half: int) -> list:
            for i in range(len(confidences)):
                probas = confidences[i]
                time = i / self.fps
                min = int(time/60)
                sec = int((time) % 60)
                for c, proba in enumerate(probas):
                    if proba > 0.5:
                        gt = str(half) + " - " + str(min).zfill(2) + \
                            ":" + str(sec).zfill(2)
                        name_action = self.classes[c]
                        actions.append({})
                        actions[-1]["label"] = name_action
                        actions[-1]["position"] = time * \
                            1000  # time in millseconds
                        actions[-1]["half"] = half
                        actions[-1]["confidence"] = proba
                        actions[-1]["gameTime"] = gt

            return actions

        my_path = dir_results
        match_name = ""
        league_names = ["england_epl", "europe_uefa-champions-league",
                        "france_ligue-1", "germany_bundesliga", "italy_serie-a"]
        for league in league_names:
            if league in dir_match:
                match_name = league
                break

        season_year = ["2014-2015", "2015-2016", "2016-2017",
                       "2017-2018", "2018-2019", "2019-2020"]
        for season in season_year:
            if season in dir_match:
                my_path = os.path.join(my_path, season)
                match_name = os.path.join(match_name, season)
                break
        match_string = copy.deepcopy(dir_match[(len(my_path)+1):-11])
        match_string = match_string.replace("_", " ")

        my_path = os.path.join(my_path, match_string)
        match_name = os.path.join(match_name, match_string)

        json_dict = {}

        # Define the half based on the name of the video that is being processed
        half = 1 if "1_224p.mkv" or "1_720p.mkv" in dir_match else 2

        json_dict["UrlLocal"] = match_name
        json_dict["predictions"] = add_action(self, [], conf, half)

        json_object = json.dumps(json_dict, indent=4)

        json_file_name = os.path.join(dir_match, f"{match_name}.json")
        # match name equals to path of last folder
        match_name = match_name.split("/")[-1]

        # Writing
        json_file_name = os.path.join(dir_results, f"{match_name}.json")

        # Writing
        with open(json_file_name, "w") as outfile:
            outfile.write(json_object)

        return json_file_name
    
    def compute_update_confidence(self, confidences: np.ndarray, displacements: np.ndarray) -> np.ndarray:
        """
        Computes the updated confidences using the displacements.

        Args:
            confidences (np.ndarray): Confidence of each class for each time step of each video.
            displacements (np.ndarray): Displacement of each class for each time step of each video.

        Returns:
            np.ndarray: Updated confidences of each class for each time step of each video.
        """
        T = confidences.shape[0]
        K = confidences.shape[1]

        displacements = np.round(displacements * self.fps).astype(int)
        output = np.zeros_like(confidences)
        classes = np.arange(K)

        for t in range(T):
            disp = displacements[t, :]
            idx = t - disp
            idx = np.clip(idx, 0, T - 1)
            output[idx, classes] = np.maximum(output[idx, classes], confidences[t, classes])
        
        return output    
    
    def merge_nms(self, confidences: np.ndarray, displacements: np.ndarray) -> np.ndarray:
        """
        Evaluates the updated confidences using the displacements and Normalized Maximum Suppression (NMS) on time series.

        Args:
            confidences (np.ndarray): Confidence of each class for each time step of each video.
            displacements (np.ndarray): Displacement of each class for each time step of each video. The displacements 
                are in seconds.
        Returns:
            np.ndarray: Updated confidences of each class for each time step of each video.
        """

        # Check the shape of the confidences and displacements
        assert confidences.shape == displacements.shape, "The shape of the confidences and displacements should be the same."

        # 1. Compute the updated confidences using the displacements
        # 2. Compute the NMS on the time series

        # 1. Compute the updated confidences using the displacements
        updated_confidences = self.compute_update_confidence(confidences, displacements)

        # 2. Compute the NMS on the time series
        nms_confidences = self.compute_nms(updated_confidences)
        return nms_confidences
       

    def compute_nms(self, confidences: np.ndarray) -> np.ndarray:
        """
        Computes the NMS on the time series for each class seperately.

        Args:
            confidences (np.ndarray): Updated confidences of each class for each time step of each video.

        Returns:
            np.ndarray: NMS confidences of each class for each time step of each video.
        """
        T = confidences.shape[0]
        K = confidences.shape[1]

        output = confidences.copy()

        for k in range(K):
            nms_window = self.nms_window[k]
            nms_window = int(nms_window * self.fps)
            preds = confidences[:, k]
            copy_preds = preds.copy()

            # Apply NMS for a window of size nms_window seconds
            max_idx = np.argmax(copy_preds)
            max_val = copy_preds[max_idx]

            while max_val > 0:
                start = max(0, max_idx - nms_window)
                end = min(T, max_idx + nms_window + 1)

                preds[start:end] = 0
                preds[max_idx] = max_val

                copy_preds[start:end] = 0
                max_idx = np.argmax(copy_preds)
                max_val = copy_preds[max_idx]
            
            output[:, k] = preds
        
        return output

In [70]:
dir_model = "/home/mdelabrassinne/DSproject/Milestone3/Time Measures"
window_size = 64
classes = ['Kick-off', 
           'Ball out of play', 
           'Throw-in', 'Foul', 
           'Clearance', 
           'Indirect free-kick', 
           'Shots on target', 
           'Corner', 
           'Shots off target', 
           'Direct free-kick', 
           'Yellow card', 
           'Substitution', 
           'Red card', 
           'Offside', 
           'Goal', 
           'Penalty', 
           'Yellow->red card']
res = Results(dir_model, features, window_size, classes, nms_windows)

conf = "confidence_window_64.pt"
disp = "disp_window_64.pt"
 
res.getResults(disp,conf)

Directory /scratch/users/mdelabrassinne/Results_Test
